In [3]:
import os

In [4]:
%pwd

'd:\\NLP\\NLP Projects\\Churn-Prediction\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\NLP\\NLP Projects\\Churn-Prediction'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [8]:
from mlProject.constants import *

In [9]:
from mlProject.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
             root_dir=config.root_dir,
             source_URL=config.source_URL,
             local_data_file=config.local_data_file,
             unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
    

In [11]:
import os
import urllib.request as request
import zipfile
from mlProject.utils.common import get_size

In [12]:
from mlProject.Logging import get_logger
logger=get_logger(__name__)

In [13]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
    
    def download(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(url=self.config.source_URL,filename=self.config.local_data_file)
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as f:
            f.extractall(unzip_path)
            


In [15]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


2026-01-07 19:36:12,067 | INFO | mlProject.utils.common | common.py:31 | yaml file: config\config.yaml loaded successfully
2026-01-07 19:36:12,081 | INFO | mlProject.utils.common | common.py:31 | yaml file: params.yaml loaded successfully
2026-01-07 19:36:12,083 | INFO | mlProject.utils.common | common.py:31 | yaml file: schema.yaml loaded successfully
2026-01-07 19:36:12,085 | INFO | mlProject.utils.common | common.py:51 | created directory at: artifacts
2026-01-07 19:36:12,087 | INFO | mlProject.utils.common | common.py:51 | created directory at: artifacts/data_ingestion
2026-01-07 19:36:13,765 | INFO | __main__ | 1236102966.py:8 | artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 1974803
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2f87d96980ae118e1d1a03044e96bae71beb6d2213d1c237da73dac3475ea816"
Strict-Transport-Security: max-age=31536